# 一、pil关系抽取
实体 - 关系
ruijin_competition

In [ ]:
# !git clone https://github.com/astronstar/ruijin_competition.git
import os,sys

base_path = os.path.dirname(os.getcwd()) #os.getcwd()
data_path = base_path + '/data'
train_data_dir = data_path + '/ruijin_competition/phase2/Demo/DataSets/ruijin_round2_train/ruijin_round2_train'
test_a_data_dir = data_path + '/ruijin_competition/phase2/Demo/DataSets/ruijin_round2_test_a/ruijin_round2_test_a'
test_b_data_dir = data_path + '/ruijin_competition/phase2/Demo/DataSets/ruijin_round2_test_b/ruijin_round2_test_b'
# train_data_dir = data_path + '/my_rujin_train'
train_data_dir

In [ ]:
import sys
sys.version

In [ ]:
# !pip uninstall keras tensorflow
# !pip install keras==2.2.4 tensorflow==1.13.1 h5py==1.5.2 sklearn
# !pip install git+https://www.github.com/keras-team/keras-contrib.git
import os
import re
import math
import zipfile
import numpy as np
from collections import Counter, defaultdict
from itertools import permutations, chain
from gensim.models import Word2Vec
from sklearn.model_selection import ShuffleSplit
from itertools import groupby
from spacy import displacy

import keras
from keras import layers
from keras import backend as K
from keras.layers import Lambda
from keras.layers import Concatenate, Dense
from keras.layers import Conv1D, MaxPool1D, Flatten
from keras.layers import Input, LSTM, Embedding, Bidirectional
from keras.models import Model
from keras.models import load_model
from keras.optimizers import Adam
from keras_contrib.layers import CRF
from keras_contrib.layers.crf import CRF, crf_loss, crf_viterbi_accuracy
from keras_contrib.utils import save_load_utils

In [ ]:
ENTITIES = [
    "Amount", "Anatomy", "Disease", "Drug",
    "Duration", "Frequency", "Level", "Method",
    "Operation", "Reason", "SideEff", "Symptom",
    "Test", "Test_Value", "Treatment", "Location"
]

RELATIONS = [
    "Test_Disease","Symptom_Disease","Treatment_Disease",
    "Drug_Disease","Anatomy_Disease","Frequency_Drug",
    "Duration_Drug","Amount_Drug","Method_Drug","SideEff-Drug",
    "Test_TestValue","Anatomy_Symptom","Symptom_Duration","Disease_Duration"
]

COLORS = [
    '#7aecec','#bfeeb7','#feca74','#ff9561',
    '#aa9cfc','#c887fb','#9cc9cc','#ffeb80',
    '#ff8197','#ff8197','#f0d0ff','#bfe1d9',
    '#bfe1d9','#e4e7d2','#e4e7d2','#e4e7d2',
    '#e4e7d2','#e4e7d2','#e4e7d2'
]

## 1.1 utils&evalu

### ent_utils

In [ ]:
COLOR_MAP = dict(zip([ent.upper() for ent in ENTITIES], COLORS[:len(ENTITIES)]))

class Sentence(object):
    def __init__(self, doc_id, offset, text, ents):
        self.text = text
        self.doc_id = doc_id
        self.offset = offset
        self.ents = ents

    def __repr__(self):
        return self.text

    def __gt__(self, other):
        return self.offset > other.offset

    def __getitem__(self, key):
        if isinstance(key, int):
            return self.text[key]
        if isinstance(key, slice):
            text = self.text[key]
            start = key.start or 0
            stop = key.stop or len(self.text)
            if start < 0:
                start += len(self.text)
            if stop < 0:
                stop += len(self.text)

            ents = self.ents.find_entities(start, stop).offset(-start)
            offset = self.offset + start
            return Sentence(self.doc_id, offset, text, ents)

    def _repr_html_(self):
        ents = []
        for ent in self.ents:
            ents.append({'start': ent.start_pos,
                         'end': ent.end_pos,
                         'label': ent.category})
        ex = {'text': self.text, 'ents': ents, 'title': None, 'settings': {}}
        return displacy.render(ex,
                               style='ent',
                               options={'colors': COLOR_MAP},
                               manual=True,
                               minify=True)

class Entity(object):
    def __init__(self, ent_id, category, start_pos, end_pos, text):
        self.ent_id = ent_id
        self.category = category
        self.start_pos = start_pos
        self.end_pos = end_pos
        self.text = text

    def __gt__(self, other):
        return self.start_pos > other.start_pos

    def offset(self, offset_val):
        return Entity(self.ent_id,
                      self.category,
                      self.start_pos + offset_val,
                      self.end_pos + offset_val,
                      self.text)

    def __repr__(self):
        return '({}, {}, ({}, {}), {})'.format(self.ent_id,
                                               self.category,
                                               self.start_pos,
                                               self.end_pos,
                                               self.text)

class Entities(object):
    def __init__(self, ents):
        self.ents = sorted(ents)
        self.ent_dict = dict(zip([ent.ent_id for ent in ents], ents))

    def __getitem__(self, key):
        if isinstance(key, int) or isinstance(key, slice):
            return self.ents[key]
        else:
            return self.ent_dict.get(key, None)

    def offset(self, offset_val):
        ents = [ent.offset(offset_val) for ent in self.ents]
        return Entities(ents)

    def vectorize(self, vec_len, cate2idx):
        res_vec = np.zeros(vec_len, dtype=int)
        for ent in self.ents:
            res_vec[ent.start_pos: ent.end_pos] = cate2idx[ent.category]
        return res_vec

    def find_entities(self, start_pos, end_pos):
        res = []
        for ent in self.ents:
            if ent.start_pos > end_pos:
                break
            sp, ep = (max(start_pos, ent.start_pos), min(end_pos, ent.end_pos))
            if ep > sp:
                new_ent = Entity(ent.ent_id, ent.category, sp, ep, ent.text[:(ep - sp)])
                res.append(new_ent)
        return Entities(res)

    def merge(self):
        merged_ents = []
        for ent in self.ents:
            if len(merged_ents) == 0:
                merged_ents.append(ent)
            elif (merged_ents[-1].end_pos == ent.start_pos and
                  merged_ents[-1].category == ent.category):
                merged_ent = Entity(ent_id=merged_ents[-1].ent_id,
                                    category=ent.category,
                                    start_pos=merged_ents[-1].start_pos,
                                    end_pos=ent.end_pos,
                                    text=merged_ents[-1].text + ent.text)
                merged_ents[-1] = merged_ent
            else:
                merged_ents.append(ent)
        return Entities(merged_ents)

class Document(object):
    def __init__(self, doc_id, text, ents=[]):
        self.doc_id = doc_id
        self.text = text
        self.ents = ents
        self.sents = self.extract_sentences()

    def extract_sentences(self):
        offset = 0
        ent_iter = iter(self.ents)
        ent = next(ent_iter, None)
        sents = []
        for text in self.text.split('。'):
            sent_ents = []
            while (ent is not None and
                   ent.start_pos >= offset and
                   ent.end_pos <= offset + len(text)):
                sent_ents.append(ent.offset(-offset))
                ent = next(ent_iter, None)
            sent = Sentence(self.doc_id, offset, text, sent_ents)
            sents.append(sent)
            offset += len(text) + 1
        return sents

    def pad(self, pad_left=0, pad_right=0, pad_val=" "):
        text = pad_left * pad_val + self.text + pad_right * pad_val
        ents = self.ents.offset(pad_left)
        return Document(self.doc_id, text, ents)

    def _repr_html_(self):
        sent = Sentence(self.doc_id, offset=0, text=self.text, ents=self.ents)
        return sent._repr_html_()

class Documents(object):
    def __init__(self, data_dir, doc_ids=None):
        self.data_dir = data_dir
        self.doc_ids = doc_ids
        if self.doc_ids is None:
            self.doc_ids = self.scan_doc_ids()

    def scan_doc_ids(self):
        doc_ids = [fname.split('.')[0] for fname in os.listdir(self.data_dir)]
        return np.unique(doc_ids)

    def read_txt_file(self, doc_id):
        fname = os.path.join(self.data_dir, doc_id + '.txt')
        with open(fname, encoding='utf-8') as f:
            text = f.read()
        return text

    def parse_entity_line(self, raw_str):
        ent_id, label, text = raw_str.strip().split('\t')
        category, pos = label.split(' ', 1)
        pos = pos.split(' ')
        ent = Entity(ent_id, category, int(pos[0]), int(pos[-1]), text)
        return ent

    def read_anno_file(self, doc_id):
        ents = []
        fname = os.path.join(self.data_dir, doc_id + '.ann')
        with open(fname, encoding='utf-8') as f:
            lines = f.readlines()

        for line in lines:
            if line.startswith('T'):
                ent = self.parse_entity_line(line)
                ents.append(ent)
        ents = Entities(ents)

        return ents

    def __len__(self):
        return len(self.doc_ids)

    def get_doc(self, doc_id):
        text = self.read_txt_file(doc_id)
        ents = self.read_anno_file(doc_id)
        doc = Document(doc_id, text, ents)
        return doc

    def __getitem__(self, key):
        if isinstance(key, int):
            doc_id = self.doc_ids[key]
            return self.get_doc(doc_id)
        if isinstance(key, str):
            doc_id = key
            return self.get_doc(doc_id)
        if isinstance(key, np.ndarray) and key.dtype == int:
            doc_ids = self.doc_ids[key]
            return Documents(self.data_dir, doc_ids=doc_ids)

class SentenceExtractor(object):
    def __init__(self, window_size=50, pad_size=10):
        self.window_size = window_size
        self.pad_size = pad_size

    def extract_doc(self, doc):
        num_sents = math.ceil(len(doc.text) / self.window_size)
        doc = doc.pad(pad_left=self.pad_size, pad_right=num_sents * self.window_size - len(doc.text) + self.pad_size)
        sents = []
        for cur_idx in range(self.pad_size, len(doc.text) - self.pad_size, self.window_size):
            sent_text = doc.text[cur_idx - self.pad_size: cur_idx + self.window_size + self.pad_size]
            ents = []
            for ent in doc.ents.find_entities(start_pos=cur_idx - self.pad_size,
                                              end_pos=cur_idx + self.window_size + self.pad_size):
                ents.append(ent.offset(-cur_idx + self.pad_size))
            sent = Sentence(doc.doc_id,
                            offset=cur_idx - 2 * self.pad_size,
                            text=sent_text,
                            ents=Entities(ents))
            sents.append(sent)
        return sents

    def __call__(self, docs):
        sents = []
        for doc in docs:
            sents += self.extract_doc(doc)
        return sents

class Dataset(object):
    def __init__(self, sentences, word2idx=None, cate2idx=None):
        self.sentences = sentences
        self.word2idx = word2idx
        self.cate2idx = cate2idx

    def build_vocab_dict(self, vocab_size=2000):
        counter = Counter()
        for sent in self.sentences:
            for char in sent.text:
                counter[char] += 1
        word2idx = dict()
        word2idx['<unk>'] = 0
        if vocab_size > 0:
            num_most_common = vocab_size - len(word2idx)
        else:
            num_most_common = len(counter)
        for char, _ in counter.most_common(num_most_common):
            word2idx[char] = word2idx.get(char, len(word2idx))
        self.word2idx = word2idx

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sent_vec, labels_vec = [], []
        sents = self.sentences[idx]
        if not isinstance(sents, list):
            sents = [sents]
        for sent in sents:
            content = [self.word2idx.get(c, 1) for c in sent.text]
            sent_vec.append(content)
            labels_vec.append(sent.ents.vectorize(vec_len=len(sent.text), cate2idx=self.cate2idx))
        return np.array(sent_vec), np.expand_dims(np.array(labels_vec), axis=-1)

def make_sentence_prediction(pred, sent, idx2ent):
    ents_vec = np.argmax(pred, axis=1)
    ents = []
    cur_idx = 0
    for label_idx, group in groupby(ents_vec):
        group = list(group)
        start_pos = cur_idx
        end_pos = start_pos + len(group)
        if label_idx > 0:
            text = sent.text[start_pos: end_pos]
            category = idx2ent[label_idx]
            ent = Entity(None, category, start_pos, end_pos, text)
            ents.append(ent)
        cur_idx = end_pos
    return Sentence(sent.doc_id, sent.offset, sent.text, Entities(ents))

def make_doc_prediction(doc_id, sents, docs):
    sents = sorted(sents)
    ents = []
    for sent in sents:
        ents += sent.ents.offset(sent.offset)
    ents = Entities(ents).merge()

    for idx, ent in enumerate(ents):
        ent.ent_id = 'T{}'.format(idx + 1)
    doc = Document(doc_id, docs[doc_id].text, ents=ents)
    return doc

def make_predictions(preds, dataset, sent_pad, docs, idx2ent):
    pred_sents = []
    for sent, pred in zip(dataset.sentences, preds):
        pred_sent = make_sentence_prediction(pred, sent, idx2ent)
        pred_sent = pred_sent[sent_pad: -sent_pad]
        pred_sents.append(pred_sent)

    docs_sents = defaultdict(list)
    for sent in pred_sents:
        docs_sents[sent.doc_id].append(sent)

    pred_docs = dict()
    for doc_id, sentences in docs_sents.items():
        doc = make_doc_prediction(doc_id, sentences, docs)
        pred_docs[doc_id] = doc

    return pred_docs


### ent_evalu

In [ ]:
class Evaluator(object):
    @staticmethod
    def check_match(ent_a, ent_b):
        return (ent_a.category == ent_b.category and
                max(ent_a.start_pos, ent_b.start_pos) < min(ent_a.end_pos, ent_b.end_pos))

    @staticmethod
    def count_intersects(ent_list_a, ent_list_b):
        num_hits = 0
        ent_list_b = ent_list_b.copy()
        for ent_a in ent_list_a:
            hit_ent = None
            for ent_b in ent_list_b:
                if Evaluator.check_match(ent_a, ent_b):
                    hit_ent = ent_b
                    break
            if hit_ent is not None:
                num_hits += 1
                ent_list_b.remove(hit_ent)
        return num_hits

    @staticmethod
    def f1_score(gt_docs, pred_docs):
        num_hits = 0
        num_preds = 0
        num_gts = 0
        for doc_id in gt_docs.doc_ids:
            gt_ents = gt_docs[doc_id].ents.ents
            pred_ents = pred_docs[doc_id].ents.ents
            num_gts += len(gt_ents)
            num_preds += len(pred_ents)
            num_hits += Evaluator.count_intersects(pred_ents, gt_ents)
        p = num_hits / num_preds
        r = num_hits / num_gts
        f = 2 * p * r / (p + r)
        return f, p, r

### re_utils

In [ ]:
class Entity(object):
    def __init__(self, ent_id, category, start_pos, end_pos, text):
        self.ent_id = ent_id
        self.category = category
        self.start_pos = start_pos
        self.end_pos = end_pos
        self.text = text
    
    def __gt__(self, other):
        return self.start_pos > other.start_pos
    
    def offset(self, offset_val):
        return Entity(self.ent_id, 
                      self.category, 
                      self.start_pos + offset_val,
                      self.end_pos + offset_val,
                      self.text)
    
    def __repr__(self):
        fmt = '({ent_id}, {category}, ({start_pos}, {end_pos}), {text})'
        return fmt.format(**self.__dict__)

class Entities(object):
    def __init__(self, ents):
        self.ents = sorted(ents)
        self.ent_dict = dict(zip([ent.ent_id for ent in ents], ents))
    
    def __getitem__(self, key):
        if isinstance(key, int) or isinstance(key, slice):
            return self.ents[key]
        else:
            return self.ent_dict.get(key, None)
        
    def __len__(self):
        return len(self.ents)
    
    def offset(self, offset_val):
        ents = [ent.offset(offset_val) for ent in self.ents]
        return Entities(ents)
    
    def vectorize(self, vec_len, cate2idx):
        res_vec = np.zeros(vec_len, dtype=int)
        for ent in self.ents:
            res_vec[ent.start_pos: ent.end_pos] = cate2idx[ent.category]
        return res_vec
    
    def find_entities(self, start_pos, end_pos):
        res = []
        for ent in self.ents:
            if ent.start_pos > end_pos:
                break
            sp, ep = (max(start_pos, ent.start_pos), min(end_pos, ent.end_pos))
            if ep > sp:
                new_ent = Entity(ent.ent_id, ent.category, sp, ep, ent.text[:(ep - sp)])
                res.append(new_ent)
        return Entities(res)
    
    def __add__(self, other):
        ents = self.ents + other.ents
        return Entities(ents)
    
    def merge(self):
        merged_ents = []
        for ent in self.ents:
            if len(merged_ents) == 0:
                merged_ents.append(ent)
            elif (merged_ents[-1].end_pos == ent.start_pos and 
                  merged_ents[-1].category == ent.category):
                merged_ent = Entity(ent_id=merged_ents[-1].ent_id, 
                                    category=ent.category,
                                    start_pos=merged_ents[-1].start_pos, 
                                    end_pos=ent.end_pos, 
                                    text=merged_ents[-1].text + ent.text)
                merged_ents[-1] = merged_ent
            else:
                merged_ents.append(ent)
        return Entities(merged_ents)

class Relation(object):
    def __init__(self, rel_id, category, ent1, ent2):
        self.rel_id = rel_id
        self.category = category
        self.ent1 = ent1
        self.ent2 = ent2
            
    @property
    def is_valid(self):
        return (isinstance(self.ent1, Entity) and
                isinstance(self.ent2, Entity) and
                [self.ent1.category, self.ent2.category] == re.split('[-_]', self.category))
    
    @property
    def start_pos(self):
        return min(self.ent1.start_pos, self.ent2.start_pos)
    
    @property
    def end_pos(self):
        return max(self.ent1.end_pos, self.ent2.end_pos)
    
    def offset(self, offset_val):
        return Relation(self.rel_id, 
                        self.category, 
                        self.ent1.offset(offset_val),
                        self.ent2.offset(offset_val))
    
    def __gt__(self, other_rel):
        return self.ent1.start_pos > other_rel.ent1.start_pos
    
    def __repr__(self):
        fmt = '({rel_id}, {category} Arg1:{ent1} Arg2:{ent2})'
        return fmt.format(**self.__dict__)

class Relations(object):
    def __init__(self, rels):
        self.rels = rels
        
    def __getitem__(self, key):
        if isinstance(key, int):
            return self.rels[key]
        elif isinstance(key, slice):
            return Relations(self.rels[key])
    
    def __add__(self, other):
        rels = self.rels + other.rels
        return Relations(rels)
    
    def find_relations(self, start_pos, end_pos):
        res = []
        for rel in self.rels:
            if start_pos <= rel.start_pos and end_pos >= rel.end_pos:
                res.append(rel)
        return Relations(res)
    
    def offset(self, offset_val): 
        return Relations([rel.offset(offset_val) for rel in self.rels])
    
    @property
    def start_pos(self):
        return min([rel.start_pos for rel in self.rels])

    @property
    def end_pos(self):
        return max([rel.end_pos for rel in self.rels])
    
    def __len__(self):
        return len(self.rels)
    
    def __repr__(self):
        return self.rels.__repr__()

class TextSpan(object):
    def __init__(self, text, ents, rels, **kwargs):
        self.text = text
        self.ents = ents
        self.rels = rels
    
    def __getitem__(self, key):
        if isinstance(key, int):
            start, stop = key, key + 1
        elif isinstance(key, slice):
            start = key.start if key.start is not None else 0
            stop = key.stop if key.stop is not None else len(self.text)
        else:
            raise ValueError('parameter should be int or slice')
        if start < 0:
            start += len(self.text)
        if stop < 0:
            stop += len(self.text)
        text = self.text[key]
        ents = self.ents.find_entities(start, stop).offset(-start)
        rels = self.rels.find_relations(start, stop).offset(-start)
        return TextSpan(text, ents, rels)
    
    def __len__(self):
        return len(self.text)

class Sentence(object):
    def __init__(self, doc_id, offset, text='', ents=[], rels=[], textspan=None):
        self.doc_id = doc_id
        self.offset = offset
        if isinstance(textspan, TextSpan):
            self.textspan = textspan
        else:
            self.textspan = TextSpan(text, ents, rels)
    
    @property
    def text(self):
        return self.textspan.text
    
    @property
    def ents(self):
        return self.textspan.ents
    
    @property
    def rels(self):
        return self.textspan.rels
    
    def abbreviate(self, max_len, ellipse_chars='$$'):
        if max_len <= len(ellipse_chars):
            return ''
        left_trim = (max_len - len(ellipse_chars)) // 2
        right_trim = max_len - len(ellipse_chars) - left_trim
        return self[:left_trim] + ellipse_chars + self[-right_trim:]
    
    def __getitem__(self, key):
        if isinstance(key, int):
            start, stop = key, key + 1
        elif isinstance(key, slice):
            start = key.start if key.start is not None else 0
            stop = key.stop if key.stop is not None else len(self.text)
        else:
            raise ValueError('parameter should be int or slice')
        if start < 0:
            start += len(self.text)
        if stop < 0:
            stop += len(self.text)
        offset = self.offset + start
        textspan = self.textspan[start: stop]
        return Sentence(self.doc_id, offset, textspan=textspan)
    
    def __gt__(self, other):
        return self.offset > other.offse
    
    def __add__(self, other):
        if isinstance(other, str):
            return Sentence(doc_id=self.doc_id, offset=self.offset, text=self.text + other, 
                            ents=self.ents, rels=self.rels)
        assert self.doc_id == other.doc_id, 'sentences should be from the same document'
        assert self.offset + len(self) <= other.offset, 'sentences should not have overlap'
        doc_id = self.doc_id
        text = self.text + other.text
        offset = self.offset
        ents = self.ents + other.ents.offset(len(self.text))
        rels = self.rels + other.rels.offset(len(self.text))
        return Sentence(doc_id=doc_id, offset=offset, text=text, ents=ents, rels=rels)
    
    def __len__(self):
        return len(self.textspan)

class Document(object):
    def __init__(self, doc_id, text, ents, rels):
        self.doc_id = doc_id
        self.textspan = TextSpan(text, ents, rels)
    
    @property
    def text(self):
        return self.textspan.text
    
    @property
    def ents(self):
        return self.textspan.ents
    
    @property
    def rels(self):
        return self.textspan.rels

class Documents(object):
    def __init__(self, data_dir, doc_ids=None):
        self.data_dir = data_dir
        self.doc_ids = doc_ids
        if self.doc_ids is None:
            self.doc_ids = self.scan_doc_ids()
    
    def scan_doc_ids(self):
        doc_ids = [fname.split('.')[0] for fname in os.listdir(self.data_dir)]
        doc_ids = [doc_id for doc_id in doc_ids if len(doc_id) > 0]
        return np.unique(doc_ids)
    
    def read_txt_file(self, doc_id):
        fname = os.path.join(self.data_dir, doc_id + '.txt')
        with open(fname, encoding='utf-8') as f:
            text = f.read()
        return text
    
    def parse_entity_line(self, raw_str):
        ent_id, label, text = raw_str.strip().split('\t')
        category, pos = label.split(' ', 1)
        pos = pos.split(' ')
        ent = Entity(ent_id, category, int(pos[0]), int(pos[-1]), text)
        return ent
    
    def parse_relation_line(self, raw_str, ents):
        rel_id, label = raw_str.strip().split('\t')
        category, arg1, arg2 = label.split(' ')
        arg1 = arg1.split(':')[1]
        arg2 = arg2.split(':')[1]
        ent1 = ents[arg1]
        ent2 = ents[arg2]
        return Relation(rel_id, category, ent1, ent2)
    
    def read_anno_file(self, doc_id):
        ents = []
        rels = []
        fname = os.path.join(self.data_dir, doc_id + '.ann')
        with open(fname, encoding='utf-8') as f:
            lines = f.readlines()
        
        for line in lines:
            if line.startswith('T'):
                ent = self.parse_entity_line(line)
                ents.append(ent)
        ents = Entities(ents)
        
        for line in lines:
            if line.startswith('R'):
                rel = self.parse_relation_line(line, ents)
                if rel.is_valid:
                    rels.append(rel)
        rels = Relations(rels)
        return ents, rels
    
    def __len__(self):
        return len(self.doc_ids)
    
    def get_doc(self, doc_id):
        text = self.read_txt_file(doc_id)
        ents, rels = self.read_anno_file(doc_id)
        doc = Document(doc_id, text, ents, rels)
        return doc
    
    def __getitem__(self, key):
        if isinstance(key, int):
            doc_id = self.doc_ids[key]
            return self.get_doc(doc_id)
        if isinstance(key, str):
            doc_id = key
            return self.get_doc(doc_id)
        if isinstance(key, np.ndarray) and key.dtype == int:
            doc_ids = self.doc_ids[key]
            return Documents(self.data_dir, doc_ids=doc_ids)

class SentenceExtractor(object):
    def __init__(self, sent_split_char, window_size, rel_types, filter_no_rel_candidates_sents=True):
        self.sent_split_char = sent_split_char
        self.window_size = window_size
        self.filter_no_rel_candidates_sents = filter_no_rel_candidates_sents
        self.rels_type_set = set()
        for rel_type in rel_types:
            self.rels_type_set.add(tuple(re.split('[-_]', rel_type)))
        
    def get_sent_boundaries(self, text):
        dot_indices = []
        for i, ch in enumerate(text):
            if ch == self.sent_split_char:
                dot_indices.append(i + 1)
        
        if len(dot_indices) <= self.window_size - 1:
            return [(0, len(text))]
        
        dot_indices = [0] + dot_indices
        if text[-1] != self.sent_split_char:
            dot_indices += [len(text)]

        boundries = []
        for i in range(len(dot_indices) - self.window_size):
            start_stop = (
                dot_indices[i],
                dot_indices[i + self.window_size]
            )
            boundries.append(start_stop)
        return boundries
            
    def has_rels_candidates(self, ents):
        ent_cates = set([ent.category for ent in ents])
        for pos_rel in permutations(ent_cates, 2):
            if pos_rel in self.rels_type_set:
                return True
        return False
        
    def extract_doc(self, doc):
        sents = []
        for start_pos, end_pos in self.get_sent_boundaries(doc.text):
            ents = []
            sent_text = doc.text[start_pos: end_pos]
            for ent in doc.ents.find_entities(start_pos=start_pos, end_pos=end_pos):
                ents.append(ent.offset(-start_pos))
            if self.filter_no_rel_candidates_sents and not self.has_rels_candidates(ents):
                continue
            rels = []
            for rel in doc.rels.find_relations(start_pos=start_pos, end_pos=end_pos):
                rels.append(rel.offset(-start_pos))
            sent = Sentence(doc.doc_id, 
                            offset=start_pos, 
                            text=sent_text, 
                            ents=Entities(ents),
                            rels=Relations(rels))
            sents.append(sent)
        return sents
        
    def __call__(self, docs):
        sents = []
        for doc in docs:
            sents += self.extract_doc(doc)
        return sents

class EntityPair(object):
    def __init__(self, doc_id, sent, from_ent, to_ent):
        self.doc_id = doc_id
        self.sent = sent
        self.from_ent = from_ent
        self.to_ent = to_ent
        
    def __repr__(self):
        fmt = 'doc {}, sent {}, {} -> {}'
        return fmt.format(self.doc_id, self.sent.text, self.from_ent, self.to_ent)

class EntityPairsExtractor(object):
    def __init__(self, allow_rel_types, max_len=150, ellipse_chars='$$', pad=10):
        self.allow_rel_types = allow_rel_types
        self.max_len = max_len
        self.pad = pad
        self.ellipse_chars = ellipse_chars
        
    def extract_candidate_rels(self, sent):
        candidate_rels = []
        for f_ent, t_ent in permutations(sent.ents, 2):
            rel_cate = (f_ent.category, t_ent.category)
            if rel_cate in self.allow_rel_types:
                candidate_rels.append((f_ent, t_ent))
        return candidate_rels
        
    def make_entity_pair(self, sent, f_ent, t_ent):
        doc_id = sent.doc_id
        if f_ent.start_pos < t_ent.start_pos:
            left_ent, right_ent = f_ent, t_ent
        else:
            left_ent, right_ent = t_ent, f_ent
        start_pos = max(0, left_ent.start_pos - self.pad)
        end_pos = min(len(sent), right_ent.end_pos + self.pad)
        res_sent = sent[start_pos: end_pos]
        
        if len(res_sent) > self.max_len:
            res_sent = res_sent.abbreviate(self.max_len)
        f_ent = res_sent.ents[f_ent.ent_id]
        t_ent = res_sent.ents[t_ent.ent_id]
        return EntityPair(doc_id, res_sent, f_ent, t_ent)
    
    def __call__(self, sents):
        samples = []
        for sent in sents:
            for f_ent, t_ent in self.extract_candidate_rels(sent.ents):
                entity_pair = self.make_entity_pair(sent, f_ent, t_ent)
                samples.append(entity_pair)
        return samples

class Dataset(object):
    def __init__(self, entity_pairs, doc_ent_pair_ids=set(), word2idx=None, cate2idx=None, max_len=150):
        self.entity_pairs = entity_pairs
        self.doc_ent_pair_ids = doc_ent_pair_ids
        self.max_len = max_len
        self.word2idx = word2idx
        self.cate2idx = cate2idx
    
    def __len__(self):
        return len(self.entity_pairs)
    
    def build_vocab_dict(self, vocab_size=2000):
        counter = Counter()
        for ent_pair in self.entity_pairs:
            for char in ent_pair.sent.text:
                counter[char] += 1
        word2idx = dict()
        word2idx['<pad>'] = 0
        word2idx['<unk>'] = 1
        if vocab_size > 0:
            num_most_common = vocab_size - len(word2idx)
        else:
            num_most_common = len(counter)
        for char, _ in counter.most_common(num_most_common):
            word2idx[char] = word2idx.get(char, len(word2idx))
        self.word2idx = word2idx
    
    def vectorize(self, ent_pair):
        sent_vec = np.zeros(self.max_len, dtype='int')
        for i, c in enumerate(ent_pair.sent.text):
            sent_vec[i] = self.word2idx.get(c, 1) 
        ents_vec = ent_pair.sent.ents.vectorize(vec_len=self.max_len, cate2idx=self.cate2idx)
        from_ent_vec = np.zeros(self.max_len, dtype='int')
        from_ent_vec[ent_pair.from_ent.start_pos: ent_pair.from_ent.end_pos] = 1
        to_ent_vec = np.zeros(self.max_len, dtype='int')
        to_ent_vec[ent_pair.to_ent.start_pos: ent_pair.to_ent.end_pos] = 1
        
        if (ent_pair.sent.doc_id, ent_pair.from_ent.ent_id, ent_pair.to_ent.ent_id) in self.doc_ent_pair_ids:
            label = 1
        else:
            label = 0
        return sent_vec, ents_vec, from_ent_vec, to_ent_vec, label
    
    def __getitem__(self, idx):
        sent_vecs, ents_vecs, from_ent_vecs, to_ent_vecs, ent_dists, labels = [], [], [], [], [], []
        entity_pairs = self.entity_pairs[idx]
        if not isinstance(entity_pairs, list):
            entity_pairs = [entity_pairs]
        for ent_pair in entity_pairs:
            sent_vec, ents_vec, from_ent_vec, to_ent_vec, label = self.vectorize(ent_pair)
            sent_vecs.append(sent_vec)
            ents_vecs.append(ents_vec)
            from_ent_vecs.append(from_ent_vec)
            to_ent_vecs.append(to_ent_vec)
            ent_dists.append(ent_pair.to_ent.start_pos - ent_pair.from_ent.end_pos)
            labels.append(label)
            
        sent_vecs = np.array(sent_vecs)
        ents_vecs = np.array(ents_vecs)
        from_ent_vecs = np.array(from_ent_vecs)
        to_ent_vecs = np.array(to_ent_vecs)
        ent_dists = np.array(ent_dists)
        labels = np.array(labels)
        
        return sent_vecs, ents_vecs, from_ent_vecs, to_ent_vecs, ent_dists, labels

def train_word_embeddings(entity_pairs, word2idx, *args, **kwargs):
    w2v_train_sents = []
    for ent_pair in entity_pairs:
        w2v_train_sents.append(list(ent_pair.sent.text))
    w2v_model = Word2Vec(w2v_train_sents, *args, **kwargs)
    word2idx.update({w: i for i, w in enumerate(w2v_model.wv.index_to_key, start=len(word2idx))})
    idx2word = {v: k for k, v in word2idx.items()}
    # vocab_size = len(word2idx)
    # w2v_embeddings = np.zeros((len(word2idx), w2v_model.vector_size))
    w2v_embeddings = np.zeros((10000, w2v_model.vector_size))
    for char, char_idx in word2idx.items():
        if char in w2v_model.wv:
            w2v_embeddings[char_idx] = w2v_model.wv[char]
    return word2idx, idx2word, w2v_embeddings


## 1.2 model

In [ ]:
def build_model():
    num_ent_classes = len(ENTITIES) + 1
    ent_emb_size = 2
    emb_size = w2v_embeddings.shape[-1]
    vocab_size = 10000#len(word2idx)
    
    inp_sent = Input(shape=(max_len,), dtype='int32')
    inp_ent = Input(shape=(max_len,), dtype='int32')
    inp_f_ent = Input(shape=(max_len,), dtype='float32')
    inp_t_ent = Input(shape=(max_len,), dtype='float32')
    inp_ent_dist = Input(shape=(1,), dtype='float32')
    f_ent = Lambda(lambda x: K.expand_dims(x))(inp_f_ent)
    t_ent = Lambda(lambda x: K.expand_dims(x))(inp_t_ent)
    
    ent_embed = Embedding(num_ent_classes, ent_emb_size)(inp_ent)
    sent_embed = Embedding(vocab_size, emb_size, weights=[w2v_embeddings], trainable=False)(inp_sent)
    
    x = Concatenate()([sent_embed, ent_embed])
    x = Conv1D(64, 1, padding='same', activation='relu')(x)
    
    f_res = layers.multiply([f_ent, x])
    t_res = layers.multiply([t_ent, x])
    
    conv = Conv1D(64, 3, padding='same', activation='relu')
    f_x = conv(x)
    t_x = conv(x)
    f_x = layers.add([f_x, f_res])
    t_x = layers.add([t_x, t_res])
    
    f_res = layers.multiply([f_ent, f_x])
    t_res = layers.multiply([t_ent, t_x])
    conv = Conv1D(64, 3, padding='same', activation='relu')
    f_x = conv(x)
    t_x = conv(x)
    f_x = layers.add([f_x, f_res])
    t_x = layers.add([t_x, t_res])

    f_res = layers.multiply([f_ent, f_x])
    t_res = layers.multiply([t_ent, t_x])
    conv = Conv1D(64, 3, padding='same', activation='relu')
    f_x = conv(x)
    t_x = conv(x)
    f_x = layers.add([f_x, f_res])
    t_x = layers.add([t_x, t_res])
    
    conv = Conv1D(64, 3, activation='relu')
    f_x = MaxPool1D(3)(conv(f_x))
    t_x = MaxPool1D(3)(conv(t_x))
    
    conv = Conv1D(64, 3, activation='relu')
    f_x = MaxPool1D(3)(conv(f_x))
    t_x = MaxPool1D(3)(conv(t_x))
    
    f_x = Flatten()(f_x)
    t_x = Flatten()(t_x)
    
    x = Concatenate()([f_x, t_x, inp_ent_dist])
    x = Dense(256, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model([inp_sent, inp_ent, inp_f_ent, inp_t_ent, inp_ent_dist], x)
    return model

def build_lstm_crf_model(num_cates, seq_len, vocab_size, model_opts=dict()):
    opts = {
        'emb_size': 256,
        'emb_trainable': True,
        'emb_matrix': None,
        'lstm_units': 256,
        'optimizer': Adam()# adam_v2.Adam()
    }
    opts.update(model_opts)

    input_seq = Input(shape=(seq_len,), dtype='int32')
    if opts.get('emb_matrix') is not None:
        embedding = Embedding(vocab_size, opts['emb_size'], weights=[opts['emb_matrix']], trainable=opts['emb_trainable'])
    else:
        embedding = Embedding(vocab_size, opts['emb_size'])
    x = embedding(input_seq)
    lstm = LSTM(opts['lstm_units'], return_sequences=True)
    x = Bidirectional(lstm)(x)
    crf = CRF(num_cates, sparse_target=True)
    output = crf(x)

    model = Model(input_seq, output)
    model.compile(opts['optimizer'], loss=crf.loss_function, metrics=[crf.accuracy])
    return model


## 1.3 实体识别训练预测


### 数据

In [ ]:
ent2idx = dict(zip(ENTITIES, range(1, len(ENTITIES) + 1)))
idx2ent = dict([(v, k) for k, v in ent2idx.items()])

docs = Documents(data_dir=train_data_dir)
rs = ShuffleSplit(n_splits=1, test_size=20, random_state=2018)
train_doc_ids, test_doc_ids = next(rs.split(docs))
train_docs, test_docs = docs[train_doc_ids], docs[test_doc_ids]
train_docs[0].text[:10]

In [ ]:
num_cates = max(ent2idx.values()) + 1
sent_len = 64
vocab_size = 10000
emb_size = 100
sent_pad = 10
sent_extrator = SentenceExtractor(window_size=sent_len, pad_size=sent_pad)
train_sents = sent_extrator(train_docs)
train_data = Dataset(train_sents, cate2idx=ent2idx)
train_data.build_vocab_dict(vocab_size=vocab_size)
vocab_size = len(train_data.word2idx)

In [ ]:
w2v_train_sents = []
for doc in docs:
    w2v_train_sents.append(list(doc.text))
w2v_model = Word2Vec(w2v_train_sents, vector_size=emb_size) # vector_size

w2v_embeddings = np.zeros((vocab_size, emb_size))
for char, char_idx in train_data.word2idx.items():
    if char in w2v_model.wv:
        w2v_embeddings[char_idx] = w2v_model.wv[char]

In [ ]:
fw = open('word2idx.txt','w')
fw.write(str(train_data.word2idx))
fw.close()
 
#读取
fr = open('word2idx.txt','r')
word2idx = fr.read()
word2idx = eval(word2idx)
fr.close()

### 训练

In [ ]:
seq_len = sent_len + 2 * sent_pad
ent_model = build_lstm_crf_model(num_cates, seq_len = seq_len, vocab_size = vocab_size, 
                             model_opts={'emb_matrix': w2v_embeddings, 'emb_size': emb_size, 'emb_trainable': False})
print(ent_model.summary())

train_x, train_y = train_data[:]
print('train_x.shape', train_x.shape)
print('train_y.shape', train_y.shape)

ent_model.fit(train_x, train_y, batch_size=64, epochs=10)
ent_model.save('ner_bs64_epo1.h5')
ent_model.to_json()

### 测试评分

In [ ]:
test_docs = Documents(data_dir = test_a_data_dir)
sent_extrator = SentenceExtractor(window_size=sent_len, pad_size=sent_pad)
test_sents = sent_extrator(test_docs)
test_data = Dataset(test_sents, cate2idx=ent2idx)
test_data.build_vocab_dict(vocab_size=vocab_size)
test_data = Dataset(test_sents, word2idx=train_data.word2idx, cate2idx=ent2idx)
test_X, _ = test_data[:]

preds = ent_model.predict(test_X, batch_size=64, verbose=True)
pred_docs = make_predictions(preds, test_data, sent_pad, test_docs, idx2ent)
f_score, precision, recall = Evaluator.f1_score(test_docs, pred_docs)
print('f_score: ', f_score)
print('precision: ', precision)
print('recall: ', recall)
# sample_doc_id = list(pred_docs.keys())[0]
# test_docs[sample_doc_id]
# pred_docs[sample_doc_id]

### 预测

In [14]:
ent2idx = dict(zip(ENTITIES, range(1, len(ENTITIES) + 1)))
idx2ent = dict([(v, k) for k, v in ent2idx.items()])
num_cates = max(ent2idx.values()) + 1
sent_len = 64
vocab_size = 10000
emb_size = 100
sent_pad = 10

model_test = load_model('ner_bs64_epo1.h5', custom_objects={"CRF": CRF, 'crf_loss': crf_loss, 'crf_viterbi_accuracy': crf_viterbi_accuracy})


txt = """主诉": "发热5天 ", "现病史": "患儿5天前（25/5）无明显诱因出现发热，热峰38.5℃，予口服退热药可降至正常，无咳嗽，无气促，无抽搐，无皮疹，次日家属发现患儿颈部肿大，伴触痛，遂至当地医院就诊，考虑急性淋巴结炎，予喜炎平肌注、奥司他韦、小儿热速清颗粒等治疗，患儿仍有反复发热，热峰升至40℃，双足可见皮疹及脱皮，27/5至广州市增城区人民医院就诊住院治疗1天，查血常规示白细胞22.9910^9/L，中性粒细胞百分比 92.8%，超敏CRP120.95mg/l，血沉105mm/h，生化示ALT263IU/L，AST410IU/L，TB41.7umol/l，降钙素原8.466mg/ml，考虑传染性单核细胞增多症待查，川崎病待查，肝功能损害，予头孢硫脒抗感染、补液、退热等治疗，患儿仍有反复高热，并出现呕吐，结膜充血、双脚蜕皮，昨家属签字出院自行至我院急诊留观，予完善相关检查及静滴舒普深治疗，患儿仍有发热，现为进一步诊治拟粘膜皮肤淋巴结综合征(川崎病)收入我科。起病以来，精神、反应一般，胃纳、睡眠欠佳，近两天结稀烂样便，小便黄，体重无明显变化。                                                                                ", "既往史": "2017.11曾因热性惊厥住院治疗，否认肝炎、结核等传染病史，否认药物敏史，否认食物过敏史，否认葡萄糖-6-磷酸脱氢酶缺陷症病史、地中海贫血，否认外伤史，否认手术史，否认输血史。 头孢硫脒，舒普深 ", "个人史": "出生史：第1胎第1产，足月。生产方式：剖宫产，出生于：医院，出生体重2.85kg，无窒息史，母妊娠期无特殊。喂养史：生后母乳喂养至4个月，按时添加钙剂，按时添加辅食。生长发育史：(精神、运动发育迟缓者及两岁内患者需填写) 于正常同龄儿童相似。 接种疫苗史：预防接种按时，家属未提供疫苗接种本： ", "家族史": "父亲、母亲体健。否认近亲婚配，否认葡萄糖-6-磷酸脱氢酶缺陷症等遗传病史，否认肝炎、结核等传染病史。   同胞情况: ", "体格检查": "T 38.8℃   P 125次/分   R 26次/分  BP 88/56mmHg  体重 15kg 一般情况:发育正常，营养中等，自主体位，表情自如，神志清晰，精神疲倦，反应一般，无脱水征，查体合作。皮肤粘膜：粘膜轻度黄染，皮肤弹性良好，皮肤灼热，未见大理石花纹。耳后及下肢见散在红色皮疹，无皮下出血点，无水肿，手足可见硬肿，足背侧有脱皮。淋 巴 结: 双侧颈部可触及数个肿大淋巴结，直径约1-2cm，伴触痛，活动欠佳。头部：头颅：正常，前囟已闭，后囟已闭。眼： 外观正常，眼睑正常，睑结膜充血，球结膜充血，无脓性分泌物，巩膜无黄染，角膜透明，双侧瞳孔等大等圆，直径约3mm，对光反射灵敏。耳：耳廓正常，无畸形，无结节，无肿胀，提拉耳廓无疼痛，乳突区无压痛，外耳道无分泌物。鼻：鼻部外形无异常，鼻唇沟双侧对称，无鼻翼扇动，鼻腔内无异常分泌物，鼻中隔无偏曲，鼻窦区无压痛。口：口腔无异味，口唇红润，口腔粘膜无出血点及溃疡，咽部充血 ，扁桃体I肿大，表面无脓点，声音稍嘶哑。颈部：颈部：颈软，无抵抗，颈静脉无怒张，肝颈静脉回流征阴性，气管居中，未触及甲状腺肿大。胸部：胸廓外形对称，无局部膨隆或凹陷，胸廓无压痛。肺脏：望视诊：呼吸平稳规则。触诊：触觉语颤对称，无胸膜摩擦感，无皮下捻发感。叩诊：清音。听诊：双肺呼吸音对称，双肺呼吸音粗，未闻及干湿啰音，未闻及支气管呼吸音，语音传导正常，无胸膜摩擦音，无捻发音 。心脏：视诊:无心前区隆起。触诊:心尖搏动位置在左侧第五肋间隙锁骨中线。                    听诊：心率125次/分，律齐，未闻及早搏音，心音有力，未闻及杂音，未闻及心包摩擦音。腹部：视诊：腹部平坦，未见腹部瘢痕，未见腹壁静脉曲张，未见胃肠蠕动波。触诊：腹部软，压之无哭闹不安，肝脾肋下未触及，Murphy征-。叩诊：鼓音，肝区无叩痛，移动性浊音阴性。听诊：肠鸣音正常，3-5次/分，无过气水音，无血管杂音。肛门、外生殖器：肛门、外生殖器无畸形，肛门外周无脱皮。脊柱四肢:脊柱生理弯曲存在，棘突无压痛、叩痛，双手有硬肿，无脱皮，四肢无畸形，双下肢无浮肿。肌肉无萎缩、无压痛，关节无畸形、肿胀、强直、活动受限，四肢肌力V级，肌张力正常。神经系统：生理反射存在。Hoffmann征、Babinski征、Oppenheim征、Gordon征、Chaddock征未引出，Kernig征、Brudzinski征阴性。 ", "体温": "头孢硫脒，舒普深 ", "专科检查": "神志清晰，反应可。全身皮肤轻度黄染，耳后及下肢可见散在红色皮疹，双侧球结膜充血，无脓性分泌物，双侧颈部可扪及多个肿大淋巴结，直径约1-2cm，杨梅舌，舌苔厚，咽充血，扁桃体I肿大，未见脓点。胸腹部未见皮疹。颈软，无抵抗。呼吸规律，平顺，26次/分，双肺呼吸音粗，未闻及明显干湿啰音。心率125次/分，律齐，未闻及病理性杂音。腹平，肠鸣音正常，肝脾肋下未及，压之无哭闹不安。四肢肌力V级，肌张力正常。双手有硬肿，足背侧有脱皮。肢端暖，CRT 1s。 ", "辅助检查结果": "我院急诊留观：2019.05.28：ABO血型(微柱凝胶法) A型   Rh(D)血型(微柱凝胶法) 阳性 ；凝血四项：纤维蛋白原 8.17g/L；|速诊生化：丙氨酸氨基转移酶 181U/L，天门冬氨酸氨基转移酶 119U/L，白蛋白 35.3g/L，总胆红素 99.6umol/L，直接胆红素 73.1umol/L，总胆汁酸 444.7 umol/L，尿酸 448 umol/L，肌酐 26  umol/L，超敏-CRP 110.13 mg/L；血气分析：钾 2.40  mmol/L，钠 123.0mmol/L，全血剩余碱 -8.30  mmol/L；血常规+CRP：快速C反应蛋白 115.5 mg/L ，白细胞 13.9*10^9/L，中性粒细胞绝对值 12.65*10^9/L，中性粒细胞百分比 91  %。2019.05.29免疫6项：免疫球蛋白A 1.58 g/L，免疫球蛋白E 938IU/ML；降钙素原 26.900  ng/ml；粪便分析：隐血试验 阳性 ；尿液分析：红细胞 8.58 个/ul，白细胞 69.96个/ul。胸片示双肺纹理增强。九项呼吸道病原学、EB病毒、咽拭子I+II类、血培养结果未回。  ", "其他信息": "姓名: 罗俊宇                      住址：广东省广州市增城区派潭镇利迳村围陇路11号  性别: 男                          病史陈述者:父母      年龄:5岁                          联系人：龙云倩 母子 广东省广州市增城区派潭镇利迳村围陇路11号 13640853793                                 民族: 汉族    """

if os.path.isfile(base_path + '/Test'):
    os.remove(base_path + '/Test')
if not os.path.exists(base_path + '/Test'):
    os.makedirs(base_path + '/Test')
with open(base_path + '/Test/{}.txt'.format('1'), 'w') as fw:
    fw.write(txt)
fw.close()
with open(base_path + '/Test/{}.ann'.format('1'), 'w') as fw:
    fw.write('')
fw.close()

fr = open('word2idx.txt','r')
train_data_word2idx = fr.read()
train_data_word2idx = eval(train_data_word2idx)
fr.close()

test_docs = Documents(data_dir = base_path + '/Test')
sent_extrator = SentenceExtractor(window_size=sent_len, pad_size=sent_pad)
test_sents = sent_extrator(test_docs)
test_data = Dataset(test_sents, cate2idx=ent2idx)
test_data.build_vocab_dict(vocab_size=vocab_size)
test_data = Dataset(test_sents, word2idx=train_data_word2idx, cate2idx=ent2idx)
test_X, _ = test_data[:]

preds = model_test.predict(test_X, batch_size=64, verbose=True)
pred_docs = make_predictions(preds, test_data, sent_pad, test_docs, idx2ent)
sample_doc_id = list(pred_docs.keys())[0]
pred_docs[sample_doc_id]

46/46 [==============================] - 2s 38ms/step


In [15]:
for i in pred_docs.keys():
    with open(base_path+'/Test/{}.ann'.format(i), 'w') as fw:
        for ent in pred_docs[i].ents.ents:
            fw.write(ent.ent_id+'\t'+ent.category+' '+str(ent.start_pos)+' '+str(ent.end_pos)+'\t'+ent.text+'\n')
    fw.close()

## 1.4 关系抽取训练预测


### 训练数据

In [ ]:
all_rel_types = set([tuple(re.split('[-_]' ,rel)) for rel in RELATIONS])
ent2idx = dict(zip(ENTITIES, range(1, len(ENTITIES) + 1)))
train_docs = Documents(train_data_dir)

# 提取训练集中所有的 relation
doc_ent_pair_ids = set()
for doc in train_docs:
    for rel in doc.rels:
        doc_ent_pair_id = (doc.doc_id, rel.ent1.ent_id, rel.ent2.ent_id)
        doc_ent_pair_ids.add(doc_ent_pair_id)

# 从文档中抽取句子
sent_extractor = SentenceExtractor(sent_split_char='。', window_size=2, rel_types=RELATIONS, filter_no_rel_candidates_sents=True)
train_sents = sent_extractor(train_docs)

# 从句子中提取 relation 候选集
max_len = 500
ent_pair_extractor = EntityPairsExtractor(all_rel_types, max_len=max_len)
train_entity_pairs = ent_pair_extractor(train_sents)

# 利用候选 relation 所在的句子训练字符级别的字向量
word2idx = {'<pad>': 0, '<unk>': 1}
word2idx, idx2word, w2v_embeddings = train_word_embeddings(entity_pairs=chain(train_entity_pairs), word2idx=word2idx, vector_size=100) #vector_size,iter=10

# 生成训练集
train_data = Dataset(train_entity_pairs, doc_ent_pair_ids, word2idx=word2idx, max_len=max_len, cate2idx=ent2idx)

### 模型训练

In [ ]:
num_ent_classes = len(ENTITIES) + 1
ent_emb_size = 2
emb_size = w2v_embeddings.shape[-1]
vocab_size = 10000#len(word2idx)
print('------->', emb_size)

tr_sent, tr_ent, tr_f_ent, tr_t_ent, tr_ent_dist, tr_y = train_data[:]
K.clear_session()
model = build_model()
model.compile('adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(x=[tr_sent, tr_ent, tr_f_ent, tr_t_ent, tr_ent_dist], y=tr_y, batch_size=64, epochs=2)
model.save('re_bs64_epo1.h5')

In [ ]:
model.save('re_bs64_epo1.h5')

### 预测

In [ ]:
def generate_submission(preds, entity_pairs, threshold):
    doc_rels = defaultdict(set)
    for p, ent_pair in zip(preds, entity_pairs):
        if p >= threshold:
            doc_id = ent_pair.doc_id
            f_ent_id = ent_pair.from_ent.ent_id
            t_ent_id = ent_pair.to_ent.ent_id
            category = ent_pair.from_ent.category + '_' + ent_pair.to_ent.category
            category = category.replace('SideEff_Drug', 'SideEff-Drug')
            doc_rels[doc_id].add((f_ent_id, t_ent_id, category))
    submits = dict()
    tot_num_rels = 0
    for doc_id, rels in doc_rels.items():
        output_str = ''
        for i, rel in enumerate(rels):
            tot_num_rels += 1
            line = 'R{}\t{} Arg1:{} Arg2:{}\n'.format(i + 1, rel[2], rel[0], rel[1])
            output_str += line
        submits[doc_id] = output_str
    # print('Total number of relations: {}. In average {} relations per doc.'.format(tot_num_rels, tot_num_rels / len(submits)))
    return submits

def output_submission(submit_file, submits, test_dir):
    dir_name = os.path.basename(submit_file)
    dir_name, _ = os.path.splitext(dir_name)
    with zipfile.ZipFile(submit_file, 'w') as zf:
        for doc_id, rels_str in submits.items():
            fname = '{}.ann'.format(doc_id)
            test_file = os.path.join(test_dir, fname)
            content = open(test_file, encoding='utf-8').read()
            content += rels_str
            zf.writestr(os.path.join(dir_name, fname), content)

            
model_re = load_model('re_bs64_epo1.h5')

max_len = 150
all_rel_types = set([tuple(re.split('[-_]' ,rel)) for rel in RELATIONS])
ent2idx = dict(zip(ENTITIES, range(1, len(ENTITIES) + 1)))
ent_pair_extractor = EntityPairsExtractor(all_rel_types, max_len=max_len)

test_docs = Documents(base_path + '/Test')
sent_extractor = SentenceExtractor(sent_split_char='。', window_size=2, rel_types=RELATIONS, filter_no_rel_candidates_sents=True)
test_sents = sent_extractor(test_docs)
test_entity_pairs = ent_pair_extractor(test_sents)
test_data = Dataset(test_entity_pairs, word2idx=word2idx, max_len=max_len, cate2idx=ent2idx)
te_sent, te_ent, te_f_ent, te_t_ent, te_ent_dist, te_y = test_data[:]

model_re.compile('adam', loss='binary_crossentropy', metrics=['acc'])
preds = model_re.predict(x=[te_sent, te_ent, te_f_ent, te_t_ent, te_ent_dist], verbose=1)


In [ ]:
submits = generate_submission(preds, test_entity_pairs, 0.5)
submits

In [ ]:
submits = generate_submission(preds, test_entity_pairs, 0.5)

for doc_id, rels_str in submits.items():
    with open(base_path + '/Test/{}.ann'.format(doc_id), 'a+') as fw:
        fw.write(rels_str)

In [ ]:
test_docs = Documents(data_dir = base_path + '/Test')

relues = []
for i in test_docs:
    #print(i.text)
    #print(i.ents.ents)
    # print(i.rels.rels)
    relue_dict = {}
    ent_list = []
    spo_list = []
    
    for e in i.ents.ents:
        e_spl = str(e).split()
        ent_list.append({
            'entity':e_spl[4].replace(')',''),
            'entity_type':e_spl[1],
            'entity_indx':e_spl[2]+e_spl[3]
              })
    for r in i.rels.rels:
        r_spl = str(r).split()
        spo_list.append({
            'predicate':r_spl[1],
            'subject':r_spl[5].replace(')',''),
            'subject_type':r_spl[3],
            'subject_indx':r_spl[4]+r_spl[5],
            'object':{'@value':r_spl[11].replace(')','')},
            'object_indx':r_spl[9]+r_spl[10],
            'object_type':{'@value':r_spl[8]}
            })
    
    relue_dict['text'] = i.text
    relue_dict['ent_list'] = ent_list
    relue_dict['spo_list'] = spo_list
    relues.append(relue_dict)
    
# print(relues)

# 二、SuperGUTScode

In [ ]:
!pip uninstall keras tensorflow
!pip install keras==2.3.1 tensorflow==2.2.0 sklearn #python3.7

In [ ]:
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold,KFold
import argparse
import numpy as np
import pickle
from glob import glob

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [ ]:
!mkdir /content/data_rujun2
!mkdir /content/model_rujun2
import os
base_path = os.getcwd()# os.path.dirname(os.getcwd())
data_path = base_path + '/data_rujun2'
model_path = base_path + '/model_rujun2'
print(base_path)

## 2.1 utils

In [ ]:
import pandas as pd
import numpy as np

def f1_score(y_true,y_pred):
    y_pred = y_pred.max(axis=1).round()
    y_true = y_true.max(axis=1)
    return 2*np.sum(y_true*y_pred)/(np.sum(y_true)+np.sum(y_pred))

def f1_score_v2(y_true,y_pred,bias):
    y_pred = y_pred.max(axis=1).round()
    y_true = y_true.max(axis=1)
    return 2 * np.sum(y_true * y_pred) / (bias + np.sum(y_pred))

def cal_total_relations(file_ids):
    all_num = 0
    for file_id in file_ids:
        label = pd.read_csv(f'{tr_path}{file_id}.ann', header=None, sep='\t')
        all_num += len(label[label[0].str.startswith('R')])
    return all_num

def split_data(dataset,file_id):
    idx = []
    for i,f_id in enumerate(dataset['file_id']):
        if f_id in file_id:
            idx.append(i)
    return idx,{k:dataset[k][idx] for k in dataset.keys()}

def concat_data(data1,data2):
    return {k:np.concatenate([data1[k],data2[k]],axis=0) for k in data1.keys()}


def filter_samples(data,distance_thres,pred,score_thres):
    condi1 = data['min_len'] < distance_thres
    condi2 = ((pred > score_thres) + (pred < (1-score_thres))).astype(bool)
    condi = condi2 * condi1
    return {k:v[condi] for k,v in data.items()}


## 2.2 model

In [ ]:
from keras.models import Sequential, Model
from keras.layers import *
import keras.backend as K
from keras.legacy import interfaces
from keras.optimizers import Optimizer


class M_Nadam(Optimizer):
  def __init__(self, lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004, multipliers=None,**kwargs):
    super(M_Nadam, self).__init__(**kwargs)
    with K.name_scope(self.__class__.__name__):
        self.iterations = K.variable(0, dtype='int64', name='iterations')
        self.m_schedule = K.variable(1., name='m_schedule')
        self.lr = K.variable(lr, name='lr')
        self.beta_1 = K.variable(beta_1, name='beta_1')
        self.beta_2 = K.variable(beta_2, name='beta_2')
    if epsilon is None:
        epsilon = K.epsilon()
    self.epsilon = epsilon
    self.schedule_decay = schedule_decay

    if multipliers is None:
        multipliers = {}
    self.lr_multipliers = multipliers

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        t = K.cast(self.iterations, K.floatx()) + 1

        # Due to the recommendations in [2], i.e. warming momentum schedule
        momentum_cache_t = self.beta_1 * (1. - 0.5 * (K.pow(K.cast_to_floatx(0.96), t * self.schedule_decay)))
        momentum_cache_t_1 = self.beta_1 * (1. - 0.5 * (K.pow(K.cast_to_floatx(0.96), (t + 1) * self.schedule_decay)))
        m_schedule_new = self.m_schedule * momentum_cache_t
        m_schedule_next = self.m_schedule * momentum_cache_t * momentum_cache_t_1
        self.updates.append((self.m_schedule, m_schedule_new))

        shapes = [K.int_shape(p) for p in params]
        ms = [K.zeros(shape) for shape in shapes]
        vs = [K.zeros(shape) for shape in shapes]

        self.weights = [self.iterations] + ms + vs

        lr = self.lr

        for p, g, m, v in zip(params, grads, ms, vs):

            matched_layer = [x for x in self.lr_multipliers.keys() if x in p.name]
            if matched_layer:
                new_lr = lr * self.lr_multipliers[matched_layer[0]]
            else:
                new_lr = lr

            # the following equations given in [1]
            g_prime = g / (1. - m_schedule_new)
            m_t = self.beta_1 * m + (1. - self.beta_1) * g
            m_t_prime = m_t / (1. - m_schedule_next)
            v_t = self.beta_2 * v + (1. - self.beta_2) * K.square(g)
            v_t_prime = v_t / (1. - K.pow(self.beta_2, t))
            m_t_bar = (1. - momentum_cache_t) * g_prime + momentum_cache_t_1 * m_t_prime

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))

            p_t = p - new_lr * m_t_bar / (K.sqrt(v_t_prime) + self.epsilon)
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))

        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'epsilon': self.epsilon,
                  'schedule_decay': self.schedule_decay}
        base_config = super(M_Nadam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class M_Adam(Optimizer):

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., amsgrad=False, multipliers=None,**kwargs):
        super(M_Adam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsgrad = amsgrad

        if multipliers is None:
            multipliers = {}
        self.lr_multipliers = multipliers

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations,
                                                      K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):

            matched_layer = [x for x in self.lr_multipliers.keys() if x in p.name]
            if matched_layer:
                new_lr = lr_t * self.lr_multipliers[matched_layer[0]]
            else:
                new_lr = lr_t


            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = p - new_lr * m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                p_t = p - new_lr * m_t / (K.sqrt(v_t) + self.epsilon)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'amsgrad': self.amsgrad}
        base_config = super(M_Adam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

class M_RMSprop(Optimizer):
    def __init__(self, lr=0.001, rho=0.9, epsilon=None, decay=0.,multipliers=None,
                 **kwargs):
        super(M_RMSprop, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.lr = K.variable(lr, name='lr')
            self.rho = K.variable(rho, name='rho')
            self.decay = K.variable(decay, name='decay')
            self.iterations = K.variable(0, dtype='int64', name='iterations')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay

        if multipliers is None:
            multipliers = {}
        self.lr_multipliers = multipliers
    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        accumulators = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = accumulators
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations,
                                                      K.dtype(self.decay))))

        for p, g, a in zip(params, grads, accumulators):

            matched_layer = [x for x in self.lr_multipliers.keys() if x in p.name]
            if matched_layer:
                new_lr = lr * self.lr_multipliers[matched_layer[0]]
            else:
                new_lr = lr

            # update accumulator
            new_a = self.rho * a + (1. - self.rho) * K.square(g)
            self.updates.append(K.update(a, new_a))
            new_p = p - new_lr * g / (K.sqrt(new_a) + self.epsilon)

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'rho': float(K.get_value(self.rho)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon}
        base_config = super(M_RMSprop, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



def f1_metric(y_true,y_pred):
    y_true = K.max(y_true, axis=1)
    y_pred = K.max(y_pred, axis=1)
    y_pred = K.round(y_pred)
    return 2*K.sum(y_pred*y_true)/(K.sum(y_true)+K.sum(y_pred)+1e-12)

def relation_attention(inputs,text):

    def share_layer(feats,f):
        return [f(x) for x in feats]

    e1_mask = Lambda(lambda x: K.expand_dims(K.cast(K.greater(x, 0), 'float32'), axis=1))(inputs['seg'])
    e2_mask = Lambda(lambda x: K.expand_dims(K.cast(K.less(x, 0), 'float32'), axis=1))(inputs['seg'])
    e1 = Lambda(lambda x: K.batch_dot(x[0], x[1]) / K.sum(x[0], keepdims=True))([e1_mask, text])
    e2 = Lambda(lambda x: K.batch_dot(x[0], x[1]) / K.sum(x[0], keepdims=True))([e2_mask, text])


    encode_feat = [e2,e1,text]
    # encode_feat = share_layer(encode_feat,Conv1D(256,1,activation='relu'))
    encode_feat = share_layer(encode_feat,Conv1D(256,1,activation='tanh'))

    atte = Lambda(lambda x:(x[2]-(x[1]+x[0])))(encode_feat)
    atte = Dense(1)(atte)
    atte = Flatten()(atte)

    atte = Activation('softmax')(atte)
    atte = Reshape((1,-1))(atte)
    text = Lambda(lambda x: K.batch_dot(x[0], x[1]))([atte,text])
    text = Flatten()(text)


    e1 = Flatten()(e1)
    e2 = Flatten()(e2)
    return text,e1,e2

def Input_moudle(cfg):
    x_in = Input((cfg['maxlen'],), name='text')
    pt_in = Input((cfg['maxlen'],), name='flag')
    pe_in = Input((cfg['maxlen'],2),name='position')
    seg_in = Input((cfg['maxlen'],), name='segment')
    len_in = Input((1,), name='min_len')
    prior_in = Input((10,), name='mask')
    pos_tag = Embedding(cfg['num_pg'], 16, name='embpos')(pt_in)
    emb = Embedding(cfg['num_word'], cfg['word_dim'], trainable=True, name='emb')



    seg = Lambda(lambda x:K.expand_dims(x))(seg_in)
    x = emb(x_in)
    x = concatenate([x, pe_in, pos_tag], axis=-1)
    x = add([x,seg])
    x = BatchNormalization(name='bn1')(x)
    x = SpatialDropout1D(0.2)(x)

    inputs = {
        'text': x_in,
        'pos_tag': pt_in,
        'position': pe_in,
        'mask': prior_in,
        'seg': seg_in,
        'len': len_in,
    }

    if cfg['use_adj_feat']:
        e1e2_in = Input((4,), name='e1e2')
        e2e1_in = Input((4,), name='e2e1')
        e1dist_in = Input((4,), name='e1dist')
        e2dist_in = Input((4,), name='e2dist')
        inputs.update({
            'e2e1': e2e1_in,
            'e1e2': e1e2_in,
            'e1dist': e1dist_in,
            'e2dist': e2dist_in,
        })

    return x,inputs

def encoder(x,encode_name,inputs,cfg):
    if encode_name == 'gru':
        x = Bidirectional(CuDNNGRU(cfg['unit1'], return_sequences=True, name='gru1'), merge_mode='sum')(x)
        x = Bidirectional(CuDNNGRU(cfg['unit2'], return_sequences=True, name='gru2'), merge_mode='sum')(x)
    elif encode_name == 'lstmgru':
        x = Bidirectional(CuDNNLSTM(cfg['unit1'], return_sequences=True, name='gru1'), merge_mode='sum')(x)
        x = Bidirectional(CuDNNGRU(cfg['unit2'], return_sequences=True, name='gru2'), merge_mode='sum')(x)
    elif encode_name == 'lstm':
        x = Bidirectional(CuDNNLSTM(cfg['unit1'], return_sequences=True, name='gru1'), merge_mode='sum')(x)
        x = Bidirectional(CuDNNLSTM(cfg['unit2'], return_sequences=True, name='gru2'), merge_mode='sum')(x)
    elif encode_name == 'grulstm':
        x = Bidirectional(CuDNNGRU(cfg['unit2'], return_sequences=True, name='gru2'), merge_mode='sum')(x)
        x = Bidirectional(CuDNNLSTM(cfg['unit2'], return_sequences=True, name='gru2'), merge_mode='sum')(x)
    else:
        raise RuntimeError("no this encode")
    return x

def rnn_model(cfg):

    def bce_loss(y_true, y_pred):
        y_true = K.max(y_true, axis=1)
        y_pred = K.max(y_pred, axis=1)
        return -K.mean(cfg['alpha']*y_true*K.log(y_pred)+(1-cfg['alpha'])*(1-y_true)*K.log(1-y_pred))


    x,inputs = Input_moudle(cfg)
    x = encoder(x,cfg['encode_name'],inputs,cfg)

    weighted_pool,e1,e2 = relation_attention(inputs,x)
    weighted_pool = Dropout(0.2)(weighted_pool)

    len_feat = BatchNormalization()(inputs['len'])
    feat = [weighted_pool,len_feat]
    if cfg['use_adj_feat']:
        prob_feat = concatenate([inputs['e2e1'], inputs['e1e2']])
        feat.append(prob_feat)
    merge = concatenate(feat,axis=1)

    output = Dense(256, activation='relu',name='d1')(merge)
    output = Dense(256, activation='relu',name='d2')(output)
    output = Dense(10,activation='sigmoid',name='d3')(output)
    output = multiply([inputs['mask'],output])


    lr_dict = {
        'emb':cfg['emb'],
        'embpos':cfg['emb'],
    }
    model = Model(list(inputs.values()), output)
    model.compile(loss=bce_loss, optimizer=M_Nadam(cfg['lr'],multipliers=lr_dict),metrics=[f1_metric])

    return model


## 2.3 训练

In [ ]:
import os
import tensorflow as tf
import datetime
import getopt
import math
import time
import numpy as np
from model import model_2c_base
from model.model_2c_ensemble import EnsembleModel
from sklearn.metrics import f1_score
from branch_stdout import *
from dataset_ss import DataSet
import relations_builder_2c as rb
import pickle


default_config = {
    "learning_rate": 0.001,
    "display_step": 60,
    "test_step": 300,
    "batch_size": 24,
    "test_batch_size": 600,
    "module_dir": 'Demo/DataSets/s{s}_{v}',
    "style": None,
    "version": "1",
    "k_flod": 1,
    "max_step": 3000,
}


def run(config, ds):
    learning_rate = config['learning_rate']
    display_step = config['display_step']
    test_step = config['test_step']
    batch_size = config['batch_size']
    test_batch_size = config['test_batch_size']
    module_dir = config['module_dir']
    style = config['style']
    version = config['version']
    k_flod = config['k_flod']
    max_step = config['max_step']
    
    module_dir = module_dir.replace("{s}", style + "_k" + str(k_flod)).replace("{v}", "v" + version)
    if not os.path.isdir(module_dir):
        os.mkdir(module_dir)
    cp_path = module_dir

    log_file = module_dir + '/train_log_' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '.log'
    sys.stdout = BranchStdout(log_file)

    reversed_relation_dict = dict(map(lambda x: (x[1], x[0]), ds.relation_dict.items()))

    m = EnsembleModel(max_train_seq_len=ds.max_train_seq_len,
                      max_test_seq_len=ds.max_test_seq_len,
                      max_train_rel_count=ds.max_train_rel_count,
                      max_test_rel_count=ds.max_test_rel_count)

    saver_best = []
    for i in range(k_flod):
        model_name = "model_" + str(i + 1) + "_" + style
        cm = model_2c_base.Model_board(max_train_seq_len=ds.max_train_seq_len,
                                       max_test_seq_len=ds.max_test_seq_len,
                                       max_train_rel_count=ds.max_train_rel_count,
                                       max_test_rel_count=ds.max_test_rel_count,
                                       ensemble_model=m,
                                       style=style,
                                       name=model_name)
        m.children_models.append(cm)
        sc_file = os.path.join(cp_path, "model_" + str(i + 1) + ".sc")
        if os.path.isfile(sc_file):
            with open(sc_file, 'rb') as f:
                sc_obj = pickle.load(f)
                sc_val = sc_obj[0]
        else:
            sc_val = 0.0
        saver_best.append([tf.train.Saver(var_list=cm.get_saving_variables()),
                           os.path.join(cp_path, model_name + "_best"),
                           sc_val])

    with m.graph.as_default():
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

        training_ops = [[optimizer.minimize(m.children_models[j].cost,
                                            var_list=m.children_models[j].get_trainable_variables())
                         for j in range(k_flod) if j != i] + [m.get_train_update_ops()]
                        for i in range(k_flod)]
        watching_ops = [[(m.children_models[j].train_cost, m.children_models[j].regularization_cost,
                          m.children_models[j].cost, m.children_models[j].pred)
                         for j in range(k_flod) if j != i]
                        for i in range(k_flod)]
        message_template = ['\n'.join(["M" + str(j + 1) + ", Minibatch Loss={" +
                                       str(j * 4 if j < i else (j - 1) * 4) + ":f}/{" +
                                       str((j * 4 if j < i else (j - 1) * 4) + 1) + ":f}/{" +
                                       str((j * 4 if j < i else (j - 1) * 4) + 2) + ":f}, F1={" +
                                       str((j * 4 if j < i else (j - 1) * 4) + 3) + ":f}"
                                       for j in range(k_flod) if j != i])
                            for i in range(k_flod)]
        test_watching_ops = [(m.children_models[j].test_cost,
                              m.children_models[j].test_pred,
                              m.children_models[j].test_score)
                             for j in range(k_flod)]

        with tf.Session(graph=m.graph) as sess:
            tf.set_random_seed(20140630)
            sess.run(tf.global_variables_initializer())

            print("learning_rate=", learning_rate)
            print("batch_size=", batch_size)
            print("seq_mask_rate", ds.seq_mask_rate)
            print("max_rel_distance=", ds.max_rel_distance)
            print("min_rel_distance=", ds.min_rel_distance)
            print("split_size=", ds.split_size)

            step = 0
            ds.reset_train()
            epoch_steps = math.ceil(ds.train_count / batch_size)
            sess_time_in_display = 0
            valid_labels = [1]

            while max_step > 0 and step < max_step:
                es = step % epoch_steps
                step += 1

                batch_data_seq, batch_data_len, \
                batch_data_rel, batch_data_rsl, batch_doc, batch_len = \
                    ds.next_train(batch_size=batch_size)
                batch_data_lbl = np.cast[np.float32](batch_data_rel[:, :, 0] > 0)

                if step % display_step == 0:
                    st = time.time()
                    w_v, _ = \
                        sess.run([watching_ops[es % k_flod], training_ops[es % k_flod]],
                                 feed_dict={
                                     m.xci: batch_data_seq[:, :, 0],
                                     m.xtc: batch_data_seq[:, :, 1],
                                     m.xsl: batch_data_len,
                                     m.rel: batch_data_rel[:, :, 1:3],
                                     m.lbl: batch_data_lbl,
                                     m.rsl: batch_data_rsl
                                 })
                    sess_time_in_step = time.time() - st
                    sess_time_in_display += sess_time_in_step
                    print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "Iter " + str(step))
                    print(message_template[es % k_flod].format(
                        *[v for sl in
                          [[w_v[i][0], w_v[i][1], w_v[i][2],
                            f1_score(batch_data_lbl.reshape([-1]), w_v[i][3].reshape([-1]),
                                     labels=valid_labels,
                                     average="micro")] for i in range(k_flod - 1)]
                          for v in sl]
                    ))
                    print("Sess Time={0:f}/{1:f}".format(sess_time_in_step, sess_time_in_display))
                    sess_time_in_display = 0
                    sys.stdout.flush()
                else:
                    st = time.time()
                    sess.run(training_ops[es % k_flod], feed_dict={m.xci: batch_data_seq[:, :, 0],
                                                                   m.xtc: batch_data_seq[:, :, 1],
                                                                   m.xsl: batch_data_len,
                                                                   m.rel: batch_data_rel[:, :, 1:3],
                                                                   m.lbl: batch_data_lbl,
                                                                   m.rsl: batch_data_rsl})
                    sess_time_in_display += time.time() - st

                if step % test_step == 0:
                    print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "Iter " + str(step))
                    for i in range(k_flod):
                        epoch_test_stat = [0.0, 0.0, 0.0, 0.0]
                        epoch_test_count = 0

                        all_data_seq, all_data_len, \
                        all_data_rel, all_data_rsl, all_doc, all_entities, all_len = \
                            ds.next_flod_test(i, k_flod, batch_size)
                        all_data_lbl = np.cast[np.float32](all_data_rel[:, :, 0] > 0)

                        j = 0
                        all_pred_score = np.zeros((all_len, ds.all_regular_rel_count), dtype=np.float32)
                        while j < len(all_data_seq):
                            batch_data_seq = all_data_seq[j:j + test_batch_size]
                            batch_data_len = all_data_len[j:j + test_batch_size]
                            batch_data_rel = all_data_rel[j:j + test_batch_size]
                            batch_data_rsl = all_data_rsl[j:j + test_batch_size]
                            batch_data_lbl = all_data_lbl[j:j + test_batch_size]

                            w_v = \
                                sess.run(test_watching_ops,
                                         feed_dict={
                                             m.test_xci: batch_data_seq[:, :, 0],
                                             m.test_xtc: batch_data_seq[:, :, 1],
                                             m.test_xsl: batch_data_len,
                                             m.test_rel: batch_data_rel[:, :, 1:3],
                                             m.test_lbl: batch_data_lbl,
                                             m.test_rsl: batch_data_rsl
                                         })

                            epoch_test_stat[0] += f1_score(batch_data_lbl.reshape([-1]), w_v[i][1].reshape([-1]),
                                                           labels=valid_labels,
                                                           average="micro")
                            epoch_test_stat[1] += w_v[i][0]
                            epoch_test_count += 1
                            if j + test_batch_size <= len(all_data_seq):
                                all_pred_score[j:j + test_batch_size, :] = w_v[i][2]
                            else:
                                all_pred_score[j:, :] = w_v[i][2]
                            j += test_batch_size

                        raw_relations_p = rb.build_raw_relations(rb.build_relations_from_data(
                            doc=all_doc,
                            filenames=ds.train_filenames,
                            entities=all_entities,
                            rel=all_data_rel,
                            rsl=all_data_rsl,
                            pred_score=all_pred_score,
                            relation_labels=reversed_relation_dict))

                        raw_relations_f = ds.get_raw_relations_of_flod_k(i, k_flod, batch_size)
                        cr = rb.compare_result(raw_relations_p, raw_relations_f)
                        model_score = cr['micro']['F1']

                        print("M" + str(i + 1),
                              ", Test Loss={0:f}".format(epoch_test_stat[1] / epoch_test_count),
                              ", F1={0:f}".format(epoch_test_stat[0] / epoch_test_count),
                              ", Micro[F1={0:f}|P={1:f}|R={2:f}]".format(cr['micro']['F1'],
                                                                         cr['micro']['P'],
                                                                         cr['micro']['R']),
                              ", Macro[F1={0:f}|P={1:f}|R={2:f}]".format(cr['macro']['F1'],
                                                                         cr['macro']['P'],
                                                                         cr['macro']['R']))

                        if model_score > saver_best[i][2]:
                            saver_best[i][0].save(sess, saver_best[i][1])
                            sc_file = os.path.join(cp_path, "model_" + str(i + 1) + ".sc")
                            with open(sc_file, 'wb') as f:
                                pickle.dump((model_score, cr['micro'], cr['macro']), f)
                            saver_best[i][2] = model_score
                            print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                                  'Got best model of model', str(i + 1), '-', model_score)

                    sys.stdout.flush()


if __name__ == "__main__":
    stdout = sys.stdout
    print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "Run Start")
    config = default_config.copy()
    data_path = 'Demo/DataSets'

    try:
        opts, args = getopt.getopt(sys.argv[1:], "d:r:s:v:k:",
                                   ["dpath=", "mdir=", "style=", "version=", "k-flod=", "max-step="])
    except getopt.GetoptError:
        print('arguments error')
        sys.exit(2)

    for opt, arg in opts:
        if opt in ("-d", "--dpath"):
            data_path = arg
        elif opt in ("-r", "--mdir"):
            config['module_dir'] = arg
        elif opt in ("-v", "--version"):
            config['version'] = arg
        elif opt in ("-k", "--k-flod"):
            config['k_flod'] = int(arg)
        elif opt in ("--max-step",):
            config['max_step'] = int(arg)

    np.random.seed(20140630)
    ts = DataSet(os.path.join(data_path, 'ruijin_round2_train/ruijin_round2_train'),
                 test_size=0,
                 seq_mask_rate=0.05,
                 min_rel_distance=0,
                 max_rel_distance=70,
                 max_split_rel_count=139,
                 all_regular_rel_count=260,
                 split_size=140,
                 worker_count=4,
                 capacity=240,
                 return_entity_detail_id=False)

    config["style"] = "1"
    run(config, ts)
    sys.stdout = stdout
    config["style"] = "6"
    run(config, ts)
    print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "Finished")

